In [1]:
import sys 
import os
# This is needed for imports to local python modules to work!
sys.path.append(os.path.abspath("/home/xilinx/jupyter_notebooks/thies/notebooks"))

from fpga_implementation import DESEncryptor, BaseFPGAEncryptor, TDESEncryptor, SDESEncryptor
from pydes_implementation import PyDES, PyTDES

In [2]:
def generic_test(f1, f2, test_vector_length=1):
    test_vectors = [
        {"message": "ABCDEFGH", "key": "12345678" * test_vector_length },
        {"message": "ABCDEFGHabcdefgh", "key": "12345678"* test_vector_length},
        {"message": "ABCDEFG", "key": "12345678"* test_vector_length},
        {"message": "", "key": "12345678"* test_vector_length},
        {"message": "ABCDEFGHIJKLMNOPQRSTUVWXYZ123456", "key": "12345678"* test_vector_length},
    ]

    for test_vector in test_vectors:
        message = test_vector["message"]
        key = test_vector["key"]
        print(f"Message: {message}, Key: {key}")
        
        f1_results = f1(message, key)
        f2_results = f2(message, key)

        assert f1_results == f2_results, f"Results do not match for message: {message}, key: {key}. f1: {f1_results}, f2: {f2_results}"
        
    print("done")

In [3]:
# Test Cases
def test_compare_des_encrypt():
    """
    Test to compare the output of the PYNQ DES implementation with the pyDes library.
    """
    fpga_encryptor = DESEncryptor(logging=False) 
    pyDes_encryptor = PyDES()
    
    generic_test(lambda m, k:fpga_encryptor.encrypt(m,k), lambda m, k:  pyDes_encryptor.encrypt(m, k).hex())

    del fpga_encryptor

def test_compare_tdes_encrypt():
    """
    Test to compare the output of the PYNQ DES implementation with the pyDes library.
    """
    fpga_encryptor = TDESEncryptor(logging=False) 
    pyDes_encryptor = PyTDES()
    
    generic_test(lambda m, k:fpga_encryptor.encrypt(m,k), lambda m, k:  pyDes_encryptor.encrypt(m, k).hex(), 3)

    del fpga_encryptor
    
test_compare_des_encrypt()
test_compare_tdes_encrypt()

Using 16x type with input: 128 bits, and output: 64 bits
Message: ABCDEFGH, Key: 12345678
Message: ABCDEFGHabcdefgh, Key: 12345678
Message: ABCDEFG, Key: 12345678
Message: , Key: 12345678
Message: ABCDEFGHIJKLMNOPQRSTUVWXYZ123456, Key: 12345678
done
Using 16x type with input: 256 bits, and output: 64 bits
Message: ABCDEFGH, Key: 123456781234567812345678
Message: ABCDEFGHabcdefgh, Key: 123456781234567812345678
Message: ABCDEFG, Key: 123456781234567812345678
Message: , Key: 123456781234567812345678
Message: ABCDEFGHIJKLMNOPQRSTUVWXYZ123456, Key: 123456781234567812345678
done


In [4]:
# Test Cases
def test_compare_des_decrypt():
    """
    Test to compare the output of the PYNQ DES implementation with the pyDes library.
    """
    fpga_encryptor = DESEncryptor() 
    pyDes_encryptor = PyDES()

    generic_test(lambda m, k:fpga_encryptor.decrypt(m,k), lambda m, k:  pyDes_encryptor.decrypt(m, k).hex())
    del fpga_encryptor

def test_compare_tdes_decrypt():
    """
    Test to compare the output of the PYNQ DES implementation with the pyDes library.
    """
    fpga_encryptor = TDESEncryptor() 
    pyDes_encryptor = PyTDES()

    generic_test(lambda m, k:fpga_encryptor.decrypt(m,k), lambda m, k:  pyDes_encryptor.decrypt(m, k).hex(), 3)
    del fpga_encryptor
    
test_compare_des_decrypt()
test_compare_tdes_decrypt()

Using 16x type with input: 128 bits, and output: 64 bits
Message: ABCDEFGH, Key: 12345678
Now sending part(0, 0): b'ABCDEFGH', 4142434445464748, with key: b'12345678', 3132333435363738
Message: ABCDEFGHabcdefgh, Key: 12345678
Now sending part(0, 0): b'ABCDEFGH', 4142434445464748, with key: b'12345678', 3132333435363738
Now sending part(1, 8): b'abcdefgh', 6162636465666768, with key: b'12345678', 3132333435363738
Message: ABCDEFG, Key: 12345678
Now sending part(0, 0): b'ABCDEFG\x00', 4142434445464700, with key: b'12345678', 3132333435363738
Message: , Key: 12345678
Message: ABCDEFGHIJKLMNOPQRSTUVWXYZ123456, Key: 12345678
Now sending part(0, 0): b'ABCDEFGH', 4142434445464748, with key: b'12345678', 3132333435363738
Now sending part(1, 8): b'IJKLMNOP', 494a4b4c4d4e4f50, with key: b'12345678', 3132333435363738
Now sending part(2, 16): b'QRSTUVWX', 5152535455565758, with key: b'12345678', 3132333435363738
Now sending part(3, 24): b'YZ123456', 595a313233343536, with key: b'12345678', 3132333

In [ ]:
fpga_encryptor = SDESEncryptor() 

In [ ]:
fpga_encryptor.input_buffer


In [ ]:
fpga_encryptor.encrypt_bytes(bytes.fromhex("28"),bytes.fromhex("00031E"))


In [ ]:
# # Performance

# def compare_des_timing():
#     """
#     Compares the execution time of the PYNQ DES implementation with the pyDes library.
#     """

#     test_vectors = [
#         {"message": "ABCDEFGH", "key": "12345678"},
#         {"message": "ABCDEFGHabcdefgh", "key": "12345678"},
#         {"message": "ABCDEFG", "key": "12345678"},
#         {"message": "", "key": "12345678"},
#         {"message": "ABCDEFGHIJKLMNOPQRSTUVWXYZ123456", "key": "12345678"},
#         {"message": "ABCDEFGHIJKLMNOPQRSTUVWXYZ123456ABCDEFGHIJKLMNOPQRSTUVWXYZ123456", "key": "12345678"},
#         {"message": "A" * 128, "key": "12345678"},
#         {"message": "A" * 128 * 4, "key": "12345678"},
# #         {"message": "A" * 2 ** 20 * 8, "key": "12345678"},
#     ]
    
    
#     largest_size = max(map(lambda x: len(x["message"]), test_vectors)) // 8
#     print("Instanting DES encryptor with size: ", largest_size)
    
#     encryptor = DESEncryptor(logging=False, max_parallel_send=largest_size)  # Create an instance of DESEncryptor
    

#     num_iterations = 10  # Number of times to repeat the encryption for timing

#     print("Timing Comparison:")
#     print(f"Number of iterations: {num_iterations}")

#     for test_vector in test_vectors:
#         message = test_vector["message"]
#         key = test_vector["key"]

#         # Time PYNQ encryption
#         start_time_pynq = time.time()
#         for _ in range(num_iterations):
#             pynq_result_hex = encryptor.encrypt(message, key)
#         end_time_pynq = time.time()
#         pynq_time = end_time_pynq - start_time_pynq

#         # Time pyDes encryption
#         start_time_pydes = time.time()
#         for _ in range(num_iterations):
#             pyDes_result_bytes = encrypt_with_pyDes(message, key)
#         end_time_pydes = time.time()
#         pydes_time = end_time_pydes - start_time_pydes

#         print(f"\nMessage: {message}, Key: {key}")
#         print(f"PYNQ Time: {pynq_time:.6f} seconds")
#         print(f"pyDes Time: {pydes_time:.6f} seconds")
#         print(f"Speed up of: {pydes_time / pynq_time:.6f}")

#         # Compare the results (optional, for functional verification)
#         pynq_result_bytes = bytes.fromhex(pynq_result_hex)
#         pyDes_result_bytes = encrypt_with_pyDes(message, key)
#         assert pynq_result_bytes == pyDes_result_bytes, f"Results do not match for message: {message}, key: {key}"

# compare_des_timing()
# print("Done")